In [1]:
import os, io
import glob
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import pywt

In [2]:
file_paths = glob.glob("../results/**/**/*.csv", recursive = True)

for idx, file in enumerate(file_paths):
    print(idx, file)

0 ../results\AL7075T6-20250331\Al7075T6_20250331_001\Al7075T6_20250331_001_secondary_data.csv
1 ../results\AL7075T6-20250331\Al7075T6_20250331_001\Al7075T6_20250331_001_primary_data.csv
2 ../results\AL7075T6-20250331\Al7075T6_20250331_002\Al7075T6_20250331_002_secondary_data.csv
3 ../results\AL7075T6-20250331\Al7075T6_20250331_002\Al7075T6_20250331_002_primary_data.csv
4 ../results\AL7075T6-20250331\Al7075T6_20250331_003\Al7075T6_20250331_003_secondary_data.csv
5 ../results\AL7075T6-20250331\Al7075T6_20250331_003\Al7075T6_20250331_003_primary_data.csv
6 ../results\AL7075T6-20250331\Al7075T6_20250331_004\Al7075T6_20250331_004_secondary_data.csv
7 ../results\AL7075T6-20250331\Al7075T6_20250331_004\Al7075T6_20250331_004_primary_data.csv
8 ../results\AL7075T6-20250331\Al7075T6_20250331_005\Al7075T6_20250331_005_secondary_data.csv
9 ../results\AL7075T6-20250331\Al7075T6_20250331_005\Al7075T6_20250331_005_primary_data.csv
10 ../results\AL7075T6-20250331\Al7075T6_20250331_006\Al7075T6_2025033

In [3]:
legends = ["A356","316SS 3D Printed","SS316 - A356 Composite"]
selected = [file_paths[i] for i in [61, 67, 201]]
#line_mode = ["solid", "longdash", "dashdot"]
line_mode = ["solid", "longdash", "dot", "dashdot", "dash", "solid"]
color = ["blue", "red", "orange"]


fig_path = "../results/mat_composite_comparison"
os.makedirs(fig_path, exist_ok=True)

file_paths[201]


'../results\\SS316_Mono\\SS316_007\\SS316_007_secondary_data.csv'

In [5]:
stress_strain_fig = go.Figure()

for idx, file in enumerate(selected):
    print(file)
    data = pd.read_csv(file)
    try:
        data.drop(labels='Unnamed: 0', axis=1, inplace=True)
    except:
        continue
    #print(data.columns)

    strain = data["Engineering Strain 3W"]
    stress_1w = data["Engineering Stress 1W"]
    stress_2w = data["Engineering Stress 2W"]

    stress_strain_fig.add_trace(go.Scatter(x=-strain, y=-stress_1w*1000, mode='lines', name=f"{legends[idx]} 1W", 
                                                        line=dict(width=3, dash="solid", color=color[idx])))
    stress_strain_fig.add_trace(go.Scatter(x=-strain, y=-stress_2w*1000, mode='lines', name=f"{legends[idx]} 2W", 
                                                        line=dict(width=3, dash="longdash", color=color[idx])))

    # Customize layout
stress_strain_fig.update_layout(width=1500, height=800, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                             title=dict(text="Engineering Stress vs Strain", x=0.5, y=0.95, xanchor="center",
                                        font=dict(size=36, color="black", family="Arial")),
                             xaxis=dict(title=dict(text="Strain (mm/mm)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridcolor="lightgrey", gridwidth=1),
                             yaxis=dict(title=dict(text="Stress (MPa)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                             legend=dict(title="Experiment", x=1.0, y=1.0, font=dict(size=16, color="black", family="Arial"),
                                         bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

stress_strain_fig.add_annotation(
        text=f"Striker Size = 18 in <br>Striker Velocity ~ 25 m/s <br>Incident Stress ~ 500 MPa",
        xref="paper", yref="paper", x=0, y=1, showarrow=False,
        font=dict(size=16, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)

# Display the interactive plot
stress_strain_fig.write_html(os.path.join(fig_path, "Eng_Stress_strain_study.html"), include_plotlyjs='cdn')


../results\SS316_Mono\SS316_011\SS316_011_initial_vel.csv
../results\A356\A356_001\A356_001_secondary_data.csv
../results\SS316_Mono\SS316_007\SS316_007_secondary_data.csv


In [7]:
true_stress_strain_fig = go.Figure()

for idx, file in enumerate(selected):
    print(file)
    data = pd.read_csv(file)
    try:
        data.drop(labels='Unnamed: 0', axis=1, inplace=True)
    except:
        continue

    strain = data["True Strain 3W"]
    stress_1w = data["Engineering Stress 1W"] * (1 + data["True Strain 3W"] )
    stress_2w = data["Engineering Stress 2W"] * (1 + data["True Strain 3W"] )

    true_stress_strain_fig.add_trace(go.Scatter(x=-strain, y=-stress_1w*1000, mode='lines', name=f"{legends[idx]} 1W", 
                                                        line=dict(width=3, dash="solid", color=color[idx])))
    true_stress_strain_fig.add_trace(go.Scatter(x=-strain, y=-stress_2w*1000, mode='lines', name=f"{legends[idx]} 2W", 
                                                        line=dict(width=3, dash="longdash", color=color[idx])))

    # Customize layout
true_stress_strain_fig.update_layout(width=1500, height=800, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                             title=dict(text="True Stress vs Strain", x=0.5, y=0.95, xanchor="center",
                                        font=dict(size=36, color="black", family="Arial")),
                             xaxis=dict(title=dict(text="True Strain (mm/mm)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridcolor="lightgrey", gridwidth=1),
                             yaxis=dict(title=dict(text="True Stress (MPa)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                             legend=dict(title="Experiment", x=1.0, y=1.0, font=dict(size=16, color="black", family="Arial"),
                                         bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

true_stress_strain_fig.add_annotation(
        text=f"Striker Size = 18 in <br>Striker Velocity ~ 25 m/s <br>Incident Stress ~ 500 MPa",
        xref="paper", yref="paper", x=0, y=1, showarrow=False,
        font=dict(size=14, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)

# Display the interactive plot
true_stress_strain_fig.write_html(os.path.join(fig_path, "True_Stress_strain_study.html"), include_plotlyjs='cdn')

../results\SS316_Mono\SS316_011\SS316_011_initial_vel.csv
../results\A356\A356_001\A356_001_secondary_data.csv
../results\SS316_Mono\SS316_007\SS316_007_secondary_data.csv


In [8]:
eng_strain_rate = go.Figure()

for idx, file in enumerate(selected):
    print(file)
    data = pd.read_csv(file)
    try:
        data.drop(labels='Unnamed: 0', axis=1, inplace=True)
    except:
        continue

    strain = data["Time"]
    stress_1w = data['Strain Rate 1W']
    stress_2w = data['Strain Rate 3W']

    eng_strain_rate.add_trace(go.Scatter(x=strain, y=stress_1w*1000, mode='lines', name=f"{legends[idx]} 1W", 
                                                        line=dict(width=3, dash="solid", color=color[idx])))
    eng_strain_rate.add_trace(go.Scatter(x=strain, y=-stress_2w*1000, mode='lines', name=f"{legends[idx]} 3W", 
                                                        line=dict(width=3, dash="longdash", color=color[idx])))
    # Customize layout
eng_strain_rate.update_layout(width=1500, height=800, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                             title=dict(text="Strain Rate vs Time", x=0.5, y=0.95, xanchor="center",
                                        font=dict(size=36, color="black", family="Arial")),
                             xaxis=dict(title=dict(text="Time (ms)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridcolor="lightgrey", gridwidth=1),
                             yaxis=dict(title=dict(text="Strain Rate (1/s)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                             legend=dict(title="Experiment", x=1.0, y=1.0, font=dict(size=20, color="black", family="Arial"),
                                         bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

eng_strain_rate.add_annotation(
        text=f"Striker Size = 18 in <br>Striker Velocity ~ 25 m/s <br>Incident Stress ~ 500 MPa",
        xref="paper", yref="paper", x=0, y=1, showarrow=False,
        font=dict(size=14, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)

# Display the interactive plot
eng_strain_rate.write_html(os.path.join(fig_path, "eng_strain_rate_study.html"), include_plotlyjs='cdn')

../results\SS316_Mono\SS316_011\SS316_011_initial_vel.csv
../results\A356\A356_001\A356_001_secondary_data.csv
../results\SS316_Mono\SS316_007\SS316_007_secondary_data.csv


In [9]:
eng_strain = go.Figure()

for idx, file in enumerate(selected):
    print(file)
    data = pd.read_csv(file)
    try:
        data.drop(labels='Unnamed: 0', axis=1, inplace=True)
    except:
        continue

    strain = data["Time"]
    stress_1w = data['Engineering Strain 1W']
    stress_2w = data['Engineering Strain 3W']

    eng_strain.add_trace(go.Scatter(x=strain, y=stress_1w, mode='lines', name=f"{legends[idx]} 1w", 
                                                        line=dict(width=3, dash="solid", color=color[idx])))
    eng_strain.add_trace(go.Scatter(x=strain, y=-stress_2w, mode='lines', name=f"{legends[idx]} 3W", 
                                                        line=dict(width=3, dash="longdash", color=color[idx])))
    # Customize layout
eng_strain.update_layout(width=1500, height=800, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                             title=dict(text="Engineering Strain vs Time", x=0.5, y=0.95, xanchor="center",
                                        font=dict(size=36, color="black", family="Arial")),
                             xaxis=dict(title=dict(text="Time (ms)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridcolor="lightgrey", gridwidth=1),
                             yaxis=dict(title=dict(text="Strain (mm/mm)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                             legend=dict(title="Experiment", x=1.0, y=1.0, font=dict(size=20, color="black", family="Arial"),
                                         bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

eng_strain.add_annotation(
        text=f"Striker Size = 18 in <br>Striker Velocity ~ 25 m/s <br>Incident Stress ~ 500 MPa",
        xref="paper", yref="paper", x=0, y=1, showarrow=False,
        font=dict(size=16, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)

# Display the interactive plot
eng_strain.write_html(os.path.join(fig_path, "eng_strain_study.html"), include_plotlyjs='cdn')

../results\SS316_Mono\SS316_011\SS316_011_initial_vel.csv
../results\A356\A356_001\A356_001_secondary_data.csv
../results\SS316_Mono\SS316_007\SS316_007_secondary_data.csv


In [10]:
eng_strain = go.Figure()

for idx, file in enumerate(selected):
    print(file)
    data = pd.read_csv(file)
    try:
        data.drop(labels='Unnamed: 0', axis=1, inplace=True)
    except:
        continue

    strain = data["Time"]
    stress_1w = np.log(1 + data['Engineering Strain 1W'])
    stress_2w = np.log(1 + data['Engineering Strain 3W'])

    eng_strain.add_trace(go.Scatter(x=strain, y=stress_1w, mode='lines', name=f"{legends[idx]} 1W", 
                                                        line=dict(width=3, dash="solid", color=color[idx])))
    eng_strain.add_trace(go.Scatter(x=strain, y=-stress_2w, mode='lines', name=f"{legends[idx]} 3W", 
                                                        line=dict(width=3, dash=line_mode[idx], color=color[idx])))
    # Customize layout
eng_strain.update_layout(width=1500, height=800, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                             title=dict(text="True Strain vs Time", x=0.5, y=0.95, xanchor="center",
                                        font=dict(size=36, color="black", family="Arial")),
                             xaxis=dict(title=dict(text="Time (ms)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridcolor="lightgrey", gridwidth=1),
                             yaxis=dict(title=dict(text="Strain (mm/mm)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                             legend=dict(title="Experiment", x=1.0, y=1.0, font=dict(size=20, color="black", family="Arial"),
                                         bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

eng_strain.add_annotation(
        text=f"Striker Size = 18 in <br>Striker Velocity ~ 25 m/s <br>Incident Stress ~ 500 MPa",
        xref="paper", yref="paper", x=0, y=1, showarrow=False,
        font=dict(size=16, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)

# Display the interactive plot
eng_strain.write_html(os.path.join(fig_path, "true_strain_study.html"), include_plotlyjs='cdn')

../results\SS316_Mono\SS316_011\SS316_011_initial_vel.csv
../results\A356\A356_001\A356_001_secondary_data.csv
../results\SS316_Mono\SS316_007\SS316_007_secondary_data.csv


In [11]:
eng_stress = go.Figure()

for idx, file in enumerate(selected):
    print(file)
    data = pd.read_csv(file)
    try:
        data.drop(labels='Unnamed: 0', axis=1, inplace=True)
    except:
        continue

    strain = data["Time"]
    stress_1w = data['Engineering Stress 1W']
    stress_2w = data['Engineering Stress 2W']

    eng_stress.add_trace(go.Scatter(x=strain, y=-stress_1w*1000, mode='lines', name=f"{legends[idx]} 1W", 
                                                        line=dict(width=3, dash="solid", color=color[idx])))
    eng_stress.add_trace(go.Scatter(x=strain, y=-stress_2w*1000, mode='lines', name=f"{legends[idx]} 2W", 
                                                        line=dict(width=3, dash="longdash", color=color[idx])))
    # Customize layout
eng_stress.update_layout(width=1500, height=800, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                             title=dict(text="Engineering Stress vs Time", x=0.5, y=0.95, xanchor="center",
                                        font=dict(size=36, color="black", family="Arial")),
                             xaxis=dict(title=dict(text="Time (ms)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridcolor="lightgrey", gridwidth=1),
                             yaxis=dict(title=dict(text="Stress (MPa)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                             legend=dict(title="Experiment", x=1.0, y=1.0, font=dict(size=20, color="black", family="Arial"),
                                         bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

eng_stress.add_annotation(
        text=f"Striker Size = 18 in <br>Striker Velocity ~ 25 m/s <br>Incident Stress ~ 500 MPa",
        xref="paper", yref="paper", x=0, y=1, showarrow=False,
        font=dict(size=16, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)

# Display the interactive plot
eng_stress.write_html(os.path.join(fig_path, "eng_stress_study.html"), include_plotlyjs='cdn')

../results\SS316_Mono\SS316_011\SS316_011_initial_vel.csv
../results\A356\A356_001\A356_001_secondary_data.csv
../results\SS316_Mono\SS316_007\SS316_007_secondary_data.csv


In [12]:
eng_stress = go.Figure()

for idx, file in enumerate(selected):
    print(file)
    data = pd.read_csv(file)
    try:
        data.drop(labels='Unnamed: 0', axis=1, inplace=True)
    except:
        continue

    strain = data["Time"]
    stress_1w = data['True Stress 1W']
    stress_2w = data['True Stress 2W']

    eng_stress.add_trace(go.Scatter(x=strain, y=-stress_1w*1000, mode='lines', name=f"{legends[idx]} 1W", 
                                                        line=dict(width=3, dash="solid", color=color[idx])))
    eng_stress.add_trace(go.Scatter(x=strain, y=-stress_2w*1000, mode='lines', name=f"{legends[idx]} 2W", 
                                                        line=dict(width=3, dash="longdash", color=color[idx])))
    # Customize layout
eng_stress.update_layout(width=1500, height=800, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                             title=dict(text="True Stress vs Time", x=0.5, y=0.95, xanchor="center",
                                        font=dict(size=36, color="black", family="Arial")),
                             xaxis=dict(title=dict(text="Time (ms)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridcolor="lightgrey", gridwidth=1),
                             yaxis=dict(title=dict(text="Stress (MPa)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                             legend=dict(title="Experiment", x=1.0, y=1.0, font=dict(size=20, color="black", family="Arial"),
                                         bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

eng_stress.add_annotation(
        text=f"Striker Size = 18 in <br>Striker Velocity ~ 25 m/s <br>Incident Stress ~ 500 MPa",
        xref="paper", yref="paper", x=0, y=1, showarrow=False,
        font=dict(size=16, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)

# Display the interactive plot
eng_stress.write_html(os.path.join(fig_path, "true_stress_study.html"), include_plotlyjs='cdn')

../results\SS316_Mono\SS316_011\SS316_011_initial_vel.csv
../results\A356\A356_001\A356_001_secondary_data.csv
../results\SS316_Mono\SS316_007\SS316_007_secondary_data.csv


In [13]:
force_interphase = go.Figure()

for idx, file in enumerate(selected):
    print(file)
    data = pd.read_csv(file)
    try:
        data.drop(labels='Unnamed: 0', axis=1, inplace=True)
    except:
        continue

    strain = data["Time"]
    stress_1w = data['Front Force']
    stress_2w = data['Back Force']

    force_interphase.add_trace(go.Scatter(x=strain, y=-stress_1w*1000, mode='lines', name=f"{legends[idx]} Front", 
                                                        line=dict(width=3, dash="solid", color=color[idx])))
    force_interphase.add_trace(go.Scatter(x=strain, y=-stress_2w*1000, mode='lines', name=f"{legends[idx]} Back", 
                                                        line=dict(width=3, dash="longdash", color=color[idx])))
    # Customize layout
force_interphase.update_layout(width=1500, height=800, plot_bgcolor="#F5F5F5", paper_bgcolor="#FFFFFF",
                             title=dict(text="Force vs Time", x=0.5, y=0.95, xanchor="center",
                                        font=dict(size=36, color="black", family="Arial")),
                             xaxis=dict(title=dict(text="Time (ms)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridcolor="lightgrey", gridwidth=1),
                             yaxis=dict(title=dict(text="Force (kN)", font=dict(family="Arial", size=24, color="black")),
                                        tickfont=dict(size=24, color="black", family="Arial"), showgrid=True,
                                        gridwidth=1, gridcolor="lightgrey", zeroline=True, zerolinewidth=2, zerolinecolor="grey"),
                             legend=dict(title="Experiment", x=1.0, y=1.0, font=dict(size=16, color="black", family="Arial"),
                                         bgcolor="#FFFFFF", bordercolor="black", borderwidth=2))

force_interphase.add_annotation(
        text=f"Striker Size = 18 in <br>Striker Velocity ~ 25 m/s <br>Incident Stress ~ 500 MPa",
        xref="paper", yref="paper", x=0, y=1, showarrow=False,
        font=dict(size=16, color="black", family="Arial"), align="left", bgcolor="white", bordercolor="black", borderwidth=2)
# Display the interactive plot
force_interphase.write_html(os.path.join(fig_path, "force_study.html"), include_plotlyjs='cdn')

../results\SS316_Mono\SS316_011\SS316_011_initial_vel.csv
../results\A356\A356_001\A356_001_secondary_data.csv
../results\SS316_Mono\SS316_007\SS316_007_secondary_data.csv
